<a href="https://colab.research.google.com/github/Tamircohen28/baswana-sen-spanner-experiments/blob/main/notebooks/04_greedy_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparison: Baswana-Sen vs. Greedy Spanner

This notebook runs a comparative study between the randomized **Baswana-Sen** algorithm and the deterministic **Greedy Spanner** baseline (Althöfer et al.).

## Objectives
1. Run both algorithms on identical random graphs.
2. Compare **Sparsity** (Number of edges).
3. Compare **Runtime**.
4. Generate the **LaTeX table** for the final report.


In [ ]:
# @title
import sys
import time
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# Add src to path to import project modules
sys.path.insert(0, str(Path().absolute().parent))

from src.graphs.erdos_renyi import generate_erdos_renyi_graph
from src.spanners.baswana_sen import build_spanner_baswana_sen
from src.spanners.greedy import build_greedy_spanner
from src.evaluation.metrics import compute_theoretical_bound

# Pandas formatting
pd.set_option('display.float_format', '{:.4f}'.format)


## 1. Run Comparison Experiments

We run the experiments on smaller graphs ($N \in \{500, 1000\}$) because the Greedy Spanner has a complexity of $O(m \cdot n)$, which can be slow for large dense graphs.


In [ ]:
# @title
def run_interactive_comparison(n_values=[500, 1000], p_values=[0.1], k_values=[2, 3], reps=3):
    results = []

    # Total iterations for progress bar
    total_ops = len(n_values) * len(p_values) * len(k_values) * reps
    pbar = tqdm(total=total_ops, desc="Running Experiments")

    for n in n_values:
        for p in p_values:
            for rep in range(reps):
                # 1. Generate Graph
                seed = 42 + rep
                G, n_orig, n_conn = generate_erdos_renyi_graph(n, p, seed)

                # Count edges in G
                m_G = sum(len(adj) for adj in G.values()) // 2

                for k in k_values:
                    # 2. Run Baswana-Sen (with explicit timing)
                    t0 = time.time()
                    bs_spanner = build_spanner_baswana_sen(G, k, seed+100)
                    t_bs = time.time() - t0
                    m_bs = sum(len(adj) for adj in bs_spanner.values()) // 2

                    # 3. Run Greedy Spanner (with explicit timing)
                    t0 = time.time()
                    greedy_spanner = build_greedy_spanner(G, k)
                    t_greedy = time.time() - t0
                    m_greedy = sum(len(adj) for adj in greedy_spanner.values()) // 2

                    # 4. Record Stats
                    results.append({
                        'n': n_conn,
                        'k': k,
                        'p': p,
                        'rep': rep,
                        'edges_bs': m_bs,
                        'edges_greedy': m_greedy,
                        'ratio_bs_greedy': m_bs / m_greedy if m_greedy > 0 else 1.0,
                        'time_bs': t_bs,
                        'time_greedy': t_greedy
                    })
                    pbar.update(1)

    return pd.DataFrame(results)


In [ ]:
# @title
# Run the experiment
# Note: Keep N small (<= 1000) for interactive use due to Greedy Spanner slowness
df_results = run_interactive_comparison(
    n_values=[500, 1000],
    p_values=[0.1],
    k_values=[2, 3],
    reps=3
)

# Display first few rows
df_results.head()


Running Experiments:   0%|          | 0/12 [00:00<?, ?it/s]

,n,k,p,rep,edges_bs,edges_greedy,ratio_bs_greedy,time_bs,time_greedy
0,500,2,0.1000,0,11508,2463,4.6724,0.0126,0.4997
1,500,3,0.1000,0,11653,499,23.3527,0.0117,0.4615
2,500,2,0.1000,1,11239,2492,4.5100,0.0115,0.4561
3,500,3,0.1000,1,11234,499,22.5130,0.0105,0.4553
4,500,2,0.1000,2,11510,2574,4.4716,0.0114,0.4721


## 2. Analyze Results

We group the results by $n$ and $k$ to get the average Edge Counts and Runtimes.


In [ ]:
# @title
# Group by N and k
summary = df_results.groupby(['n', 'k', 'p'])[['edges_bs', 'edges_greedy', 'ratio_bs_greedy', 'time_bs', 'time_greedy']].mean()
summary


edges_bs  edges_greedy  ratio_bs_greedy  time_bs  time_greedy
n    k p                                                                     
500  2 0.1000 11419.0000     2509.6667           4.5513   0.0119       0.4760
     3 0.1000 11499.0000      499.0000          23.0441   0.0109       0.4557
1000 2 0.1000 46526.6667     5947.3333           7.8321   0.0737       3.5192
     3 0.1000 46946.6667      999.0000          46.9937   0.0674       3.4498

## 3. Generate LaTeX Table

The code below generates the formatted LaTeX table used in **Section 6.3** of the report.


In [ ]:
# @title
def generate_latex_table(summary_df):
    print("\\begin{table}[h]")
    print("\\centering")
    print("\\begin{tabular}{@{}lccccc@{}}")
    print("\\toprule")
    print("$n$ & $k$ & $p$ & $|E_{BS}|$ (Avg) & $|E_{Greedy}|$ (Avg) & Ratio (BS/Greedy) \\\\ \\midrule")

    for index, row in summary_df.iterrows():
        n, k, p = index
        edges_bs = int(row['edges_bs'])
        edges_gr = int(row['edges_greedy'])
        ratio = row['ratio_bs_greedy']

        print(f"{n} & {k} & {p} & {edges_bs} & {edges_gr} & {ratio:.2f} \\\\")

    print("\\bottomrule")
    print("\\end{tabular}")
    print("\\caption{Comparison of Spanner Sizes ($|E_{BS}|$ vs $|E_{Greedy}|$)}")
    print("\\label{tab:comparison}")
    print("\\end{table}")

generate_latex_table(summary)


\begin{table}[h]
\centering
\begin{tabular}{@{}lccccc@{}}
\toprule
$n$ & $k$ & $p$ & $|E_{BS}|$ (Avg) & $|E_{Greedy}|$ (Avg) & Ratio (BS/Greedy) \\ \midrule
500 & 2 & 0.1 & 11419 & 2509 & 4.55 \\
500 & 3 & 0.1 & 11499 & 499 & 23.04 \\
1000 & 2 & 0.1 & 46526 & 5947 & 7.83 \\
1000 & 3 & 0.1 & 46946 & 999 & 46.99 \\
\bottomrule
\end{tabular}
\caption{Comparison of Spanner Sizes ($|E_{BS}|$ vs $|E_{Greedy}|$)}
\label{tab:comparison}
\end{table}
